# Table of Contents

1. [Import](#Imports)
2. [CNN](#CNN)
3. [LSTMS](#LSTMS)
4. [CNN-LSTM](#CNN-LSTM)

In [58]:
X__cnn_lstm

array([[[[ 6.76791743],
         [ 2.31135705]],

        [[ 0.44771629],
         [-5.39892344]]],


       [[[ 2.31135705],
         [ 0.44771629]],

        [[-5.39892344],
         [-1.35742137]]],


       [[[ 0.44771629],
         [-5.39892344]],

        [[-1.35742137],
         [ 4.19405614]]],


       ...,


       [[[-0.11106836],
         [ 4.71840146]],

        [[ 3.16894811],
         [ 1.73194233]]],


       [[[ 4.71840146],
         [ 3.16894811]],

        [[ 1.73194233],
         [ 0.54088648]]],


       [[[ 3.16894811],
         [ 1.73194233]],

        [[ 0.54088648],
         [ 0.4562716 ]]]])

# Imports

In [36]:
import pandas as pd
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [2]:
df = pd.read_csv('../data/clean.csv')

In [3]:
# Change date column to be datetime dtype
df['TimePeriod'] = pd.to_datetime(df['TimePeriod'])

In [4]:
# Set TimePeriod to be in the index
df.set_index('TimePeriod',inplace = True)

In [5]:
# Sort the index
df.sort_index(inplace=True)

In [6]:
df

,gdp,consumption,investment,Exports,Imports,government_spending,Residual,gdp_percent_delta,consumption_pct_delta,investment_pct_delta,...,unemployment_rate,employed,Unemployed,civilian_labor_force,civilian_labor_force_participation_rate,unemployment_rate_percent_delta,employed_percent_delta,Unemployed_percent_delta,civilian_labor_force_percent_delta,civilian_labor_force_participation_rate_percent_delta
TimePeriod,,,,,,,,,,,,,,,,,,,,,
1947-03-31,2033061.0,1274952.0,218091.0,97759.0,60271.0,524076.0,-28008.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-06-30,2027639.0,1296163.0,201386.0,96526.0,61432.0,523632.0,-24154.0,-1.062506,6.822599,-27.294690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-09-30,2023452.0,1300388.0,195461.0,91627.0,54482.0,523527.0,-15586.0,-0.823430,1.310237,-11.259195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1947-12-31,2055103.0,1300801.0,233284.0,81245.0,57419.0,519285.0,-19169.0,6.405174,0.127100,102.908083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-03-31,2086017.0,1307283.0,257220.0,77539.0,64749.0,527020.0,-32899.0,6.154156,2.008182,47.801596,...,4.0,57671.0,2399.0,60070.0,58.5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-30,19020599.0,13212778.0,3429949.0,2531422.0,3482850.0,3300261.0,-18532.0,1.491131,3.704973,-5.772133,...,3.7,157148.0,5985.0,163133.0,63.0,-0.026316,0.002597,-0.033742,0.001215,0.000000
2019-09-30,19141744.0,13301349.0,3445703.0,2536562.0,3486787.0,3317715.0,-24572.0,2.572102,2.708456,1.849925,...,3.5,158298.0,5753.0,164051.0,63.2,-0.054054,0.007318,-0.038764,0.005627,0.003175
2019-12-31,19253959.0,13353687.0,3413313.0,2557846.0,3419332.0,3337460.0,-37347.0,2.365628,1.583229,-3.707360,...,3.5,158803.0,5753.0,164556.0,63.2,0.000000,0.003190,0.000000,0.003078,0.000000


# CNN

In [27]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        #gather input and out output parts of the patterns
        seq_x , seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [28]:
seq = df.loc['1948-06-30':'2013-03-31', 'gdp_percent_delta'].dropna().values

In [29]:
n_steps = 3

In [30]:
X, y = split_sequence(seq, n_steps)

In [31]:
n_features = 1
X = X.reshape((X.shape[0],X.shape[1], n_features))

In [32]:
# Define model
model = Sequential()
model.add(Conv1D(64, 2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
#fit model
model.fit(X, y, epochs=1_000, verbose=0)


In [33]:
x_input = df.loc['2013-03-31':'2020-06-30', 'gdp_percent_delta'].values
x_input = x_input.reshape((10, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[3.4741871]
 [2.0258505]
 [4.285545 ]
 [2.2569735]
 [3.0109541]
 [2.7604039]
 [4.259362 ]
 [2.5415587]
 [3.3049254]
 [0.7178205]]


# LSTMS

In [79]:
seq = df.loc['1948-06-30':'2013-03-31', 'gdp_percent_delta'].dropna().values 

In [80]:
n_steps_lstm = 3

In [81]:
X_lstm, y_lstm = split_sequence(seq, n_steps_lstm)

In [82]:
n_features_lstm = 1
X_lstm = X_lstm.reshape((X_lstm.shape[0], X_lstm.shape[1], n_features_lstm))

In [85]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps_lstm, n_features_lstm)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
#fit Model
model.fit(X_lstm, y_lstm, epochs=200, verbose=0)

In [89]:
x_input_lstm = df.loc['2012-06-30':'2020-06-30', 'gdp_percent_delta'].values
x_input_lstm = x_input_lstm.reshape((11,n_steps_lstm, n_features_lstm))
yhat_lstm = model.predict(x_input_lstm, verbose=0)

In [90]:
yhat_lstm

array([[ 1.7666976],
       [ 2.6121287],
       [ 2.6799362],
       [ 3.189288 ],
       [ 2.1896102],
       [ 2.511311 ],
       [ 2.7460217],
       [ 3.806623 ],
       [ 2.5874903],
       [ 2.7322152],
       [-7.4535465]], dtype=float32)

# CNN-LSTM

In [63]:
seq = df.loc['1948-06-30':'2013-03-31', 'gdp_percent_delta'].dropna().values 

In [64]:
n_steps_cnn_lstm = 4

In [65]:
X_cnn_lstm, y_cnn_lstm = split_sequence(seq, n_steps_cnn_lstm)

In [66]:
n_features_cnn_lstm = 1
X_cnn_lstm = X_cnn_lstm.reshape((X_cnn_lstm.shape[0],X_cnn_lstm.shape[1], n_features_cnn_lstm))

In [67]:
#reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features_cnn_lstm = 1
n_seq_cnn_lstm = 2
n_steps_cnn_lstm = 2
X_cnn_lstm = X_cnn_lstm.reshape((X_cnn_lstm.shape[0], n_seq_cnn_lstm, n_steps_cnn_lstm, n_features_cnn_lstm))

In [68]:
#define
model = Sequential()
model.add(TimeDistributed(Conv1D(64, 1, activation='relu'), input_shape=(None, n_steps_cnn_lstm, n_features_cnn_lstm)))
model.add(TimeDistributed(MaxPooling1D()))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
#fit model
model.fit(X_cnn_lstm, y_cnn_lstm, epochs=1000, verbose=0)

In [77]:
x_input_cnn_lstm = df.loc['2012-09-30':'2020-06-30', 'gdp_percent_delta'].values
x_input_cnn_lstm = x_input_cnn_lstm.reshape((8,n_seq_cnn_lstm, n_steps_cnn_lstm, n_features_cnn_lstm))
yhat_cnn_lstm = model.predict(x_input_cnn_lstm, verbose=0)

In [78]:
yhat_cnn_lstm

array([[ 0.9615568 ],
       [-0.49217567],
       [ 3.4408948 ],
       [ 2.832585  ],
       [ 1.3688174 ],
       [ 3.1949167 ],
       [ 2.6424377 ],
       [35.137306  ]], dtype=float32)